In [1]:
import numpy as np

from vlmc import VLMC, BICSolver, ContextAlgorithmSolver
from vlmc.utils import count_word_ocurrences

import re
import glob
import copy
glob.glob('./*')

['./OrderEst_ex3',
 './OrderEst_ex4',
 './examples.ipynb',
 './OrderEst_ex2',
 './vlmc',
 './OrderEst_ex1']

In [2]:
def parse_txt_input(input_path):
    """
        Função para ler e decodificar arquivos txt. Lê-se o arquivo e analisa-se cada linha
        para obter o respectivo valor do estado. Retorna-se um array com a sequência de estados observados (observed_sequence).
    """

    observed_sequence = []
    with open(
        input_path,
        mode='r',
        encoding='utf-8'
    ) as f:
        lines = f.readlines()
        # Use regular expression to obtain sequence from input
        observed_sequence = [
            int(i) for i in re.search('[0-9]{1000,100000}', lines[-1]).group(0)
        ]
        
    return observed_sequence

# BIC

### 1st Order sample

In [3]:
N_SAMPLES = 100000

# Declare state space and mapping from state number to zero-indexed position in transition matrix
STATE_SPACE = [
    0,
    1
]
STATE_TO_INDEX_MAP = {
    s: i for i, s in enumerate(STATE_SPACE)
}

Q = np.array([
    [0.2, 0.8],
    [0.5, 0.5]
])

X = [1, 1]
for i in range(len(X)-1, N_SAMPLES):
    current_s = X[i]
    X.append(
        np.random.choice(
            a=STATE_SPACE,
            p=Q[current_s,:]
        )
    )

In [4]:
chain = VLMC(
    vocabulary=STATE_SPACE,
    max_order=5
)
sample=copy.deepcopy(X)
len(sample), sample[:10]

(100001, [1, 1, 1, 1, 1, 0, 0, 1, 0, 0])

In [5]:
chain.fit(
    X=sample,
    method='bic',
    njobs=1
)
chain.show_tree().show()

root
├── 0
└── 1



In [6]:
leaves = chain.get_leaves()
{n.word: n.transition_probabilities for n in leaves}

{'0': {'0': 0.19758515704285826, '1': 0.8024148429571417},
 '1': {'0': 0.5008689720142284, '1': 0.4991959978559943}}

In [7]:
Q[0], Q[1]

(array([0.2, 0.8]), array([0.5, 0.5]))

### 2nd Order sample

In [8]:
N_SAMPLES = 100000

# Declare state space and mapping from state number to zero-indexed position in transition matrix
STATE_SPACE = [
    0,
    1
]
STATE_TO_INDEX_MAP = {
    s: i for i, s in enumerate(STATE_SPACE)
}

Q = np.array([
    [[0.2, 0.8],
    [0.1, 0.9]],
    [[0.4, 0.6],
    [0.8, 0.2]]
])

X = [1, 1]
for i in range(len(X)-1, N_SAMPLES):
    # import pdb;pdb.set_trace()
    current_s = X[i-1:i+1]
    X.append(
        np.random.choice(
            a=STATE_SPACE,
            p=Q[tuple(current_s)]
        )
    )

In [9]:
chain = VLMC(
    vocabulary=STATE_SPACE,
    max_order=5
)
sample=copy.deepcopy(X)
len(sample), sample[:10]

(100001, [1, 1, 0, 1, 1, 0, 1, 1, 0, 1])

In [10]:
chain.fit(
    X=sample,
    method='bic',
    njobs=10
)
chain.show_tree().show()

root
├── 0
│   ├── 00
│   └── 10
└── 1
    ├── 01
    └── 11



In [11]:
leaves = chain.get_leaves()
{n.word: n.transition_probabilities for n in leaves}

{'00': {'0': 0.2031024531024531, '1': 0.7968975468975469},
 '10': {'0': 0.4009947720011618, '1': 0.5990415335463258},
 '01': {'0': 0.09835172814406042, '1': 0.9016845774034272},
 '11': {'0': 0.7999227003349653, '1': 0.20010950785879927}}

In [12]:
Q[0,0], Q[1,0], Q[0,1], Q[1, 1]

(array([0.2, 0.8]), array([0.4, 0.6]), array([0.1, 0.9]), array([0.8, 0.2]))

### 4th order complex tree

In [13]:
def dirichlet_gen_fn():
    is_valid=False
    while not is_valid:
        probs = np.random.dirichlet(
            alpha = [0.5, 0.5, 0.5]
        )
        if max(probs)<0.8 and min(probs)>0.2:
            is_valid=True
    return probs

In [14]:
N_SAMPLES=1000000
MAX_DEPTH = 4
STATE_SPACE = [
    0,
    1,
    2
]

contexts = [
    '0',
    '11',
    '21',
    '001',
    '101',
    '0201',
    '1201',
    '2201',
    '02',
    '12',
    '022',
    '122',
    '222'
]

context_probs = {
    c: dirichlet_gen_fn() for c in contexts
}
context_probs

{'0': array([0.55407684, 0.24465266, 0.2012705 ]),
 '11': array([0.30001177, 0.23839307, 0.46159516]),
 '21': array([0.4900239 , 0.21367809, 0.29629801]),
 '001': array([0.36018688, 0.34436199, 0.29545113]),
 '101': array([0.34143145, 0.30443438, 0.35413417]),
 '0201': array([0.20187306, 0.28412786, 0.51399908]),
 '1201': array([0.27580266, 0.28451471, 0.43968263]),
 '2201': array([0.21080697, 0.32627197, 0.46292106]),
 '02': array([0.49719438, 0.23394048, 0.26886514]),
 '12': array([0.33337722, 0.3309611 , 0.33566169]),
 '022': array([0.27054195, 0.3783884 , 0.35106966]),
 '122': array([0.51398286, 0.24911844, 0.23689871]),
 '222': array([0.32818303, 0.26778335, 0.40403362])}

In [15]:
def context_identification_fn(full_word, ctxts):
    context_found = False
    context=None
    word = copy.deepcopy(full_word)
    while not context_found:
        if len(word)==0:
            import pdb;pdb.set_trace()
        if word in ctxts:
            context=word
            context_found=True
        else:
            word = word[1:]
            
            
    return context

In [16]:
sample = [0, 0, 1, 1]
i=MAX_DEPTH
while len(sample) < N_SAMPLES:
    word = ''.join(
        [str(x) for x in sample[i-MAX_DEPTH:i]]
    )
    ctxt = context_identification_fn(
        full_word=word,
        ctxts=contexts
    )
    new_state = np.random.choice(
        a=STATE_SPACE,
        p=context_probs[ctxt]
    )
    sample.append(new_state)
    i+=1
    

In [17]:
chain = VLMC(
    vocabulary=STATE_SPACE,
    max_order=5
)
len(sample), sample[:10]

(1000000, [0, 0, 1, 1, 2, 2, 1, 0, 0, 0])

In [18]:
chain.fit(
    X=sample,
    method='bic',
    njobs=10
)
chain.show_tree().show()

root
├── 0
├── 1
│   ├── 01
│   │   ├── 001
│   │   ├── 101
│   │   └── 201
│   │       ├── 0201
│   │       ├── 1201
│   │       └── 2201
│   ├── 11
│   └── 21
└── 2
    ├── 02
    ├── 12
    └── 22
        ├── 022
        ├── 122
        └── 222



In [19]:
leaves = chain.get_leaves()
{n.word: n.transition_probabilities for n in leaves}

{'0': {'0': 0.5535925075560052,
  '1': 0.24523614948567277,
  '2': 0.2011756668525941},
 '001': {'0': 0.35859020996872804,
  '1': 0.34341693790286554,
  '2': 0.2980088071989278},
 '101': {'0': 0.34441685256113475,
  '1': 0.30746243528768047,
  '2': 0.34812071215118484},
 '0201': {'0': 0.19991173874669022,
  '1': 0.28649602824360104,
  '2': 0.5135922330097087},
 '1201': {'0': 0.26836447402098795,
  '1': 0.2826977220373688,
  '2': 0.44893780394164323},
 '2201': {'0': 0.19941423659747867,
  '1': 0.34521838787724435,
  '2': 0.45536737552527695},
 '11': {'0': 0.29865163168032843,
  '1': 0.24078005708420624,
  '2': 0.4605823713847841},
 '21': {'0': 0.4873809614626642,
  '1': 0.21619559827592782,
  '2': 0.296423440261408},
 '02': {'0': 0.4948577692282893,
  '1': 0.2351133225151258,
  '2': 0.27002890825658493},
 '12': {'0': 0.33234644093240484,
  '1': 0.32947958800717236,
  '2': 0.33817397106042285},
 '022': {'0': 0.2671628129104151,
  '1': 0.37983046125681535,
  '2': 0.35300672583276954},
 '1

In [20]:
context_probs

{'0': array([0.55407684, 0.24465266, 0.2012705 ]),
 '11': array([0.30001177, 0.23839307, 0.46159516]),
 '21': array([0.4900239 , 0.21367809, 0.29629801]),
 '001': array([0.36018688, 0.34436199, 0.29545113]),
 '101': array([0.34143145, 0.30443438, 0.35413417]),
 '0201': array([0.20187306, 0.28412786, 0.51399908]),
 '1201': array([0.27580266, 0.28451471, 0.43968263]),
 '2201': array([0.21080697, 0.32627197, 0.46292106]),
 '02': array([0.49719438, 0.23394048, 0.26886514]),
 '12': array([0.33337722, 0.3309611 , 0.33566169]),
 '022': array([0.27054195, 0.3783884 , 0.35106966]),
 '122': array([0.51398286, 0.24911844, 0.23689871]),
 '222': array([0.32818303, 0.26778335, 0.40403362])}

# Context Algorithm

### 1st Order sample

In [21]:
N_SAMPLES = 100000

# Declare state space and mapping from state number to zero-indexed position in transition matrix
STATE_SPACE = [
    0,
    1
]
STATE_TO_INDEX_MAP = {
    s: i for i, s in enumerate(STATE_SPACE)
}

Q = np.array([
    [0.2, 0.8],
    [0.5, 0.5]
])

X = [1, 1]
for i in range(len(X)-1, N_SAMPLES):
    current_s = X[i]
    X.append(
        np.random.choice(
            a=STATE_SPACE,
            p=Q[current_s,:]
        )
    )

In [22]:
chain = VLMC(
    vocabulary=STATE_SPACE,
    max_order=5
)
sample=copy.deepcopy(X)
len(sample), sample[:10]

(100001, [1, 1, 1, 0, 1, 1, 0, 1, 0, 1])

In [23]:
chain.fit(
    X=sample,
    method='context',
    njobs=1
)
chain.show_tree().show()

root
├── 0
└── 1



In [24]:
leaves = chain.get_leaves()
{n.word: n.transition_probabilities for n in leaves}

{'0': {'0': 0.2031107758732801, '1': 0.7969152339584363},
 '1': {'0': 0.49779850200653136, '1': 0.5022502396464605}}

In [25]:
Q[0], Q[1]

(array([0.2, 0.8]), array([0.5, 0.5]))

### 2nd Order sample

In [26]:
N_SAMPLES = 100000

# Declare state space and mapping from state number to zero-indexed position in transition matrix
STATE_SPACE = [
    0,
    1
]
STATE_TO_INDEX_MAP = {
    s: i for i, s in enumerate(STATE_SPACE)
}

Q = np.array([
    [[0.2, 0.8],
    [0.1, 0.9]],
    [[0.4, 0.6],
    [0.8, 0.2]]
])

X = [1, 1]
for i in range(len(X)-1, N_SAMPLES):
    # import pdb;pdb.set_trace()
    current_s = X[i-1:i+1]
    X.append(
        np.random.choice(
            a=STATE_SPACE,
            p=Q[tuple(current_s)]
        )
    )

In [27]:
chain = VLMC(
    vocabulary=STATE_SPACE,
    max_order=5
)
sample=copy.deepcopy(X)
len(sample), sample[:10]

(100001, [1, 1, 0, 1, 1, 1, 0, 1, 1, 0])

In [28]:
chain.fit(
    X=sample,
    method='context',
    njobs=10
)
chain.show_tree().show()

root
├── 0
│   ├── 00
│   └── 10
└── 1
    ├── 01
    └── 11



In [29]:
leaves = chain.get_leaves()
{n.word: n.transition_probabilities for n in leaves}

{'00': {'0': 0.20195107746068724, '1': 0.7980489225393127},
 '10': {'0': 0.39697255015571814, '1': 0.6030636633591656},
 '01': {'0': 0.10194473617499004, '1': 0.8980914786513599},
 '11': {'0': 0.7991492926884285, '1': 0.20088293107337352}}

In [30]:
Q[0,0], Q[1,0], Q[0,1], Q[1, 1]

(array([0.2, 0.8]), array([0.4, 0.6]), array([0.1, 0.9]), array([0.8, 0.2]))

### 4th order complex tree

In [31]:
def dirichlet_gen_fn():
    is_valid=False
    while not is_valid:
        probs = np.random.dirichlet(
            alpha = [0.5, 0.5, 0.5]
        )
        if max(probs)<0.8 and min(probs)>0.2:
            is_valid=True
    return probs

In [32]:
N_SAMPLES=1000000
MAX_DEPTH = 4
STATE_SPACE = [
    0,
    1,
    2
]

contexts = [
    '0',
    '11',
    '21',
    '001',
    '101',
    '0201',
    '1201',
    '2201',
    '02',
    '12',
    '022',
    '122',
    '222'
]

context_probs = {
    c: dirichlet_gen_fn() for c in contexts
}
context_probs

{'0': array([0.39312836, 0.38670895, 0.22016269]),
 '11': array([0.28967432, 0.20439054, 0.50593514]),
 '21': array([0.4254214 , 0.28260473, 0.29197387]),
 '001': array([0.42451565, 0.25854842, 0.31693593]),
 '101': array([0.26252204, 0.36970818, 0.36776977]),
 '0201': array([0.39302932, 0.25560387, 0.35136681]),
 '1201': array([0.44238172, 0.20994152, 0.34767676]),
 '2201': array([0.36975051, 0.37785987, 0.25238962]),
 '02': array([0.35037085, 0.35627778, 0.29335137]),
 '12': array([0.26384975, 0.51128641, 0.22486384]),
 '022': array([0.20725858, 0.42719387, 0.36554756]),
 '122': array([0.23757598, 0.41598716, 0.34643686]),
 '222': array([0.54635549, 0.24831904, 0.20532547])}

In [33]:
def context_identification_fn(full_word, ctxts):
    context_found = False
    context=None
    word = copy.deepcopy(full_word)
    while not context_found:
        if len(word)==0:
            import pdb;pdb.set_trace()
        if word in ctxts:
            context=word
            context_found=True
        else:
            word = word[1:]
            
            
    return context

In [34]:
sample = [0, 0, 1, 1]
i=MAX_DEPTH
while len(sample) < N_SAMPLES:
    word = ''.join(
        [str(x) for x in sample[i-MAX_DEPTH:i]]
    )
    ctxt = context_identification_fn(
        full_word=word,
        ctxts=contexts
    )
    new_state = np.random.choice(
        a=STATE_SPACE,
        p=context_probs[ctxt]
    )
    sample.append(new_state)
    i+=1
    

In [35]:
chain = VLMC(
    vocabulary=STATE_SPACE,
    max_order=5
)
len(sample), sample[:10]

(1000000, [0, 0, 1, 1, 2, 1, 2, 1, 2, 2])

In [36]:
chain.fit(
    X=sample,
    method='context',
    njobs=10
)
chain.show_tree().show()

root
├── 0
├── 1
│   ├── 01
│   │   ├── 001
│   │   ├── 101
│   │   └── 201
│   │       ├── 0201
│   │       ├── 1201
│   │       └── 2201
│   ├── 11
│   └── 21
└── 2
    ├── 02
    ├── 12
    └── 22
        ├── 022
        ├── 122
        └── 222



In [37]:
leaves = chain.get_leaves()
{n.word: n.transition_probabilities for n in leaves}

{'0': {'0': 0.3926518734085122,
  '1': 0.38606766096762457,
  '2': 0.22128606206452697},
 '001': {'0': 0.425359380759307,
  '1': 0.26103943973461113,
  '2': 0.3136196092886104},
 '101': {'0': 0.2627454909819639,
  '1': 0.3671743486973948,
  '2': 0.3700801603206413},
 '0201': {'0': 0.4032804911573353,
  '1': 0.25547512141482637,
  '2': 0.34124438742783836},
 '1201': {'0': 0.4427782751417487,
  '1': 0.20784840346165323,
  '2': 0.34937332139659805},
 '2201': {'0': 0.3658202507638816,
  '1': 0.382467600885049,
  '2': 0.2517121483510694},
 '11': {'0': 0.28957151038683815,
  '1': 0.20473785599000363,
  '2': 0.5057010464934659},
 '21': {'0': 0.42445579977669656,
  '1': 0.2826662754781872,
  '2': 0.29287792474511626},
 '02': {'0': 0.35254991717353096,
  '1': 0.3551548412387299,
  '2': 0.2922952415877392},
 '12': {'0': 0.2635301494089401,
  '1': 0.5101007783945803,
  '2': 0.22636907219647964},
 '022': {'0': 0.21263248972528662,
  '1': 0.4220203331170236,
  '2': 0.36534717715768983},
 '122': {'0

In [38]:
context_probs

{'0': array([0.39312836, 0.38670895, 0.22016269]),
 '11': array([0.28967432, 0.20439054, 0.50593514]),
 '21': array([0.4254214 , 0.28260473, 0.29197387]),
 '001': array([0.42451565, 0.25854842, 0.31693593]),
 '101': array([0.26252204, 0.36970818, 0.36776977]),
 '0201': array([0.39302932, 0.25560387, 0.35136681]),
 '1201': array([0.44238172, 0.20994152, 0.34767676]),
 '2201': array([0.36975051, 0.37785987, 0.25238962]),
 '02': array([0.35037085, 0.35627778, 0.29335137]),
 '12': array([0.26384975, 0.51128641, 0.22486384]),
 '022': array([0.20725858, 0.42719387, 0.36554756]),
 '122': array([0.23757598, 0.41598716, 0.34643686]),
 '222': array([0.54635549, 0.24831904, 0.20532547])}

# BCT

### 1st Order sample

In [39]:
N_SAMPLES = 100000

# Declare state space and mapping from state number to zero-indexed position in transition matrix
STATE_SPACE = [
    0,
    1
]
STATE_TO_INDEX_MAP = {
    s: i for i, s in enumerate(STATE_SPACE)
}

Q = np.array([
    [0.2, 0.8],
    [0.5, 0.5]
])

X = [1, 1]
for i in range(len(X)-1, N_SAMPLES):
    current_s = X[i]
    X.append(
        np.random.choice(
            a=STATE_SPACE,
            p=Q[current_s,:]
        )
    )

In [40]:
chain = VLMC(
    vocabulary=STATE_SPACE,
    max_order=5
)
sample=copy.deepcopy(X)
len(sample), sample[:10]

(100001, [1, 1, 0, 1, 0, 0, 1, 1, 1, 0])

In [41]:
chain.fit(
    X=sample,
    method='bct',
    njobs=1
)
chain.show_tree().show()

/home/leonardo/git/markov_chains/vlmc/vlmc/bct_solver.py:33: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  prod_vectors = np.array([
/home/leonardo/git/markov_chains/vlmc/vlmc/bct_solver.py:90: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  prod_vectors = np.array([


root
├── 0
└── 1



In [42]:
leaves = chain.get_leaves()
{n.word: n.transition_probabilities for n in leaves}

{'0': {'0': 0.2020065498778396, '1': 0.7980194417008889},
 '1': {'0': 0.4990572478137902, '1': 0.5009915152303241}}

In [43]:
Q[0], Q[1]

(array([0.2, 0.8]), array([0.5, 0.5]))

### 2nd Order sample

In [44]:
N_SAMPLES = 100000

# Declare state space and mapping from state number to zero-indexed position in transition matrix
STATE_SPACE = [
    0,
    1
]
STATE_TO_INDEX_MAP = {
    s: i for i, s in enumerate(STATE_SPACE)
}

Q = np.array([
    [[0.2, 0.8],
    [0.1, 0.9]],
    [[0.4, 0.6],
    [0.8, 0.2]]
])

X = [1, 1]
for i in range(len(X)-1, N_SAMPLES):
    # import pdb;pdb.set_trace()
    current_s = X[i-1:i+1]
    X.append(
        np.random.choice(
            a=STATE_SPACE,
            p=Q[tuple(current_s)]
        )
    )

In [45]:
chain = VLMC(
    vocabulary=STATE_SPACE,
    max_order=5
)
sample=copy.deepcopy(X)
len(sample), sample[:10]

(100001, [1, 1, 0, 1, 1, 0, 0, 1, 1, 1])

In [46]:
chain.fit(
    X=sample,
    method='bct',
    njobs=10
)
chain.show_tree().show()

root
├── 0
│   ├── 00
│   └── 10
└── 1
    ├── 01
    └── 11



In [47]:
leaves = chain.get_leaves()
{n.word: n.transition_probabilities for n in leaves}

{'00': {'0': 0.19727101175787487, '1': 0.8027289882421251},
 '10': {'0': 0.40086988039144617, '1': 0.5991663646248641},
 '01': {'0': 0.10090612540775644, '1': 0.8991301196085538},
 '11': {'0': 0.7992783040144339, '1': 0.20075391455635028}}

In [48]:
Q[0,0], Q[1,0], Q[0,1], Q[1, 1]

(array([0.2, 0.8]), array([0.4, 0.6]), array([0.1, 0.9]), array([0.8, 0.2]))

### 4th order complex tree

In [49]:
def dirichlet_gen_fn():
    is_valid=False
    while not is_valid:
        probs = np.random.dirichlet(
            alpha = [0.5, 0.5, 0.5]
        )
        if max(probs)<0.8 and min(probs)>0.2:
            is_valid=True
    return probs

In [50]:
N_SAMPLES=1000000
MAX_DEPTH = 4
STATE_SPACE = [
    0,
    1,
    2
]

contexts = [
    '0',
    '11',
    '21',
    '001',
    '101',
    '0201',
    '1201',
    '2201',
    '02',
    '12',
    '022',
    '122',
    '222'
]

context_probs = {
    c: dirichlet_gen_fn() for c in contexts
}
context_probs

{'0': array([0.36847725, 0.42965646, 0.20186629]),
 '11': array([0.31288397, 0.38579749, 0.30131855]),
 '21': array([0.34256738, 0.28733514, 0.37009748]),
 '001': array([0.2008652 , 0.30623691, 0.49289789]),
 '101': array([0.21379502, 0.38992394, 0.39628104]),
 '0201': array([0.22261146, 0.52047779, 0.25691075]),
 '1201': array([0.34960812, 0.41727646, 0.23311541]),
 '2201': array([0.28975376, 0.31290698, 0.39733926]),
 '02': array([0.49034094, 0.20173961, 0.30791944]),
 '12': array([0.20623951, 0.45626518, 0.33749531]),
 '022': array([0.49990591, 0.27428841, 0.22580568]),
 '122': array([0.38482736, 0.26769893, 0.34747371]),
 '222': array([0.35130201, 0.39288887, 0.25580912])}

In [51]:
def context_identification_fn(full_word, ctxts):
    context_found = False
    context=None
    word = copy.deepcopy(full_word)
    while not context_found:
        if len(word)==0:
            import pdb;pdb.set_trace()
        if word in ctxts:
            context=word
            context_found=True
        else:
            word = word[1:]
            
            
    return context

In [52]:
sample = [0, 0, 1, 1]
i=MAX_DEPTH
while len(sample) < N_SAMPLES:
    word = ''.join(
        [str(x) for x in sample[i-MAX_DEPTH:i]]
    )
    ctxt = context_identification_fn(
        full_word=word,
        ctxts=contexts
    )
    new_state = np.random.choice(
        a=STATE_SPACE,
        p=context_probs[ctxt]
    )
    sample.append(new_state)
    i+=1
    

In [53]:
chain = VLMC(
    vocabulary=STATE_SPACE,
    max_order=5
)
len(sample), sample[:10]

(1000000, [0, 0, 1, 1, 1, 0, 0, 1, 1, 2])

In [54]:
chain.fit(
    X=sample,
    method='bct',
    njobs=10
)
chain.show_tree().show()

root
├── 0
├── 1
│   ├── 01
│   │   ├── 001
│   │   ├── 101
│   │   └── 201
│   │       ├── 0201
│   │       ├── 1201
│   │       └── 2201
│   ├── 11
│   └── 21
└── 2
    ├── 02
    ├── 12
    └── 22
        ├── 022
        ├── 122
        └── 222



In [55]:
leaves = chain.get_leaves()
{n.word: n.transition_probabilities for n in leaves}

{'0': {'0': 0.3683614715906837,
  '1': 0.43023309034653845,
  '2': 0.2014115407030852},
 '001': {'0': 0.2012988256929944,
  '1': 0.3066033217754449,
  '2': 0.4921170092526963},
 '101': {'0': 0.21825582882230085,
  '1': 0.3890286006564085,
  '2': 0.39271557052129064},
 '0201': {'0': 0.225007286505392,
  '1': 0.52083940542116,
  '2': 0.254153308073448},
 '1201': {'0': 0.34297658862876257,
  '1': 0.42558528428093645,
  '2': 0.231438127090301},
 '2201': {'0': 0.2894818108825891,
  '1': 0.31332221604595145,
  '2': 0.39719597307145943},
 '11': {'0': 0.31198803010609677,
  '1': 0.38525073300486656,
  '2': 0.3027687936402382},
 '21': {'0': 0.34196786374845867,
  '1': 0.2868950369913687,
  '2': 0.37113709926017263},
 '02': {'0': 0.4882438492303963,
  '1': 0.2028693491697976,
  '2': 0.3088868015998061},
 '12': {'0': 0.20566338739696888,
  '1': 0.45679340600904017,
  '2': 0.33754320659399095},
 '022': {'0': 0.5021335033596547,
  '1': 0.26739908774339105,
  '2': 0.23046740889695425},
 '122': {'0':

In [56]:
context_probs

{'0': array([0.36847725, 0.42965646, 0.20186629]),
 '11': array([0.31288397, 0.38579749, 0.30131855]),
 '21': array([0.34256738, 0.28733514, 0.37009748]),
 '001': array([0.2008652 , 0.30623691, 0.49289789]),
 '101': array([0.21379502, 0.38992394, 0.39628104]),
 '0201': array([0.22261146, 0.52047779, 0.25691075]),
 '1201': array([0.34960812, 0.41727646, 0.23311541]),
 '2201': array([0.28975376, 0.31290698, 0.39733926]),
 '02': array([0.49034094, 0.20173961, 0.30791944]),
 '12': array([0.20623951, 0.45626518, 0.33749531]),
 '022': array([0.49990591, 0.27428841, 0.22580568]),
 '122': array([0.38482736, 0.26769893, 0.34747371]),
 '222': array([0.35130201, 0.39288887, 0.25580912])}